# Simple implementation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)

X_test = torch.tensor(X_test, dtype=torch.float32)

# Define a two-layer neural network using nn.Sequential
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 32),  # Input layer to hidden layer
    nn.ReLU(),                        # ReLU activation function
    nn.Linear(32, len(np.unique(y_train)))  # Hidden layer to output layer
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
batch_size = 32

loss_array = []
for epoch in range(num_epochs):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss_array.append(loss.detach().numpy())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, axis=1)
    accuracy = (predicted.numpy() == y_test).sum().item() / y_test.shape[0]

print(f"Accuracy: {accuracy * 100:.2f}%")


import matplotlib.pyplot as plt
plt.plot(np.asarray(loss_array).squeeze(),'-')

## Minibatch implementation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define a two-layer neural network using nn.Sequential
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 32),  # Input layer to hidden layer
    nn.ReLU(),                        # ReLU activation function
    nn.Linear(32, len(np.unique(y_train)))  # Hidden layer to output layer
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
batch_size = 32

loss_array = np.zeros(num_epochs)
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss_array[epoch] += loss.detach().numpy()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
accuracy = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, axis=1)
        accuracy += (predicted.numpy() == y_batch.numpy()).sum().item()/ y_batch.shape[0]

print(f"Accuracy: {accuracy * 100:.2f}%")


import matplotlib.pyplot as plt
plt.plot(np.asarray(loss_array).squeeze(),'-')

In [ ]:
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 32),  # Input layer to hidden layer
    nn.ReLU(),                        # ReLU activation function
    nn.Dropout(0.2),                  # Dropout layer with a dropout probability
    nn.Linear(32, len(np.unique(y_train)))  # Hidden layer to output layer
)
